In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

In [2]:
df = pd.read_csv("./UCL_Preds_Machine/spi_matches.csv")

In [3]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
1,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
2,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
3,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
4,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05


In [4]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [5]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-08-12,1843,French Ligue 1,Bastia,Paris Saint-Germain,51.16,85.68,0.0463,0.8380,...,32.4,67.7,0.0,1.0,0.97,0.63,0.43,0.45,0.00,1.05
1,2016,2016-08-12,1843,French Ligue 1,AS Monaco,Guingamp,68.85,56.48,0.5714,0.1669,...,53.7,22.9,2.0,2.0,2.45,0.77,1.75,0.42,2.10,2.10
2,2016,2016-08-13,2411,Barclays Premier League,Hull City,Leicester City,53.57,66.81,0.3459,0.3621,...,38.1,22.2,2.0,1.0,0.85,2.77,0.17,1.25,2.10,1.05
3,2016,2016-08-13,2411,Barclays Premier League,Everton,Tottenham Hotspur,68.02,73.25,0.3910,0.3401,...,31.9,48.0,1.0,1.0,0.73,1.11,0.88,1.81,1.05,1.05
4,2016,2016-08-13,2411,Barclays Premier League,Crystal Palace,West Bromwich Albion,55.19,58.66,0.4214,0.2939,...,43.6,34.6,0.0,1.0,1.11,0.68,0.84,1.60,0.00,1.05


In [71]:
last_played = datetime(2020,8,8)
cl = df[(df['league']== "UEFA Champions League")]
all = df[(df['league']== "UEFA Champions League") | (df['league']=="Barclays Premier League") | (df['league']=="Italy Serie A") | (df['league']=="German Bundesliga") | (df['league']=="French Ligue 1")]
old = all[all['date']<last_played]
new = cl[cl['date']>last_played]



In [72]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])


In [73]:
new = pd.concat([new,teams],axis=1)
old = pd.concat([old,teamsold],axis=1)

In [41]:
new.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2', 'team1_Atalanta',
       'team1_Barcelona', 'team1_Manchester City', 'team1_RB Leipzig',
       'team2_Atletico Madrid', 'team2_Bayern Munich', 'team2_Lyon',
       'team2_Paris Saint-Germain'],
      dtype='object')

In [42]:
for column in new.columns:
    print("'{}' ,".format(column))

'season' ,
'date' ,
'league_id' ,
'league' ,
'team1' ,
'team2' ,
'spi1' ,
'spi2' ,
'prob1' ,
'prob2' ,
'probtie' ,
'proj_score1' ,
'proj_score2' ,
'importance1' ,
'importance2' ,
'score1' ,
'score2' ,
'xg1' ,
'xg2' ,
'nsxg1' ,
'nsxg2' ,
'adj_score1' ,
'adj_score2' ,
'team1_Atalanta' ,
'team1_Barcelona' ,
'team1_Manchester City' ,
'team1_RB Leipzig' ,
'team2_Atletico Madrid' ,
'team2_Bayern Munich' ,
'team2_Lyon' ,
'team2_Paris Saint-Germain' ,


In [90]:
X_train = old[['spi1' ,
'spi2' ,
'prob1' ,
'prob2' ,
'probtie' ,
'proj_score1' ,
'proj_score2' ,

'score1','score2',

'team1_Atalanta' ,
'team1_Barcelona' ,
'team1_Manchester City' ,
'team1_RB Leipzig' ,
'team2_Atletico Madrid' ,
'team2_Bayern Munich' ,
'team2_Lyon' ,
'team2_Paris Saint-Germain']].dropna()
X_test = new[['spi1' ,
'spi2' ,
'prob1' ,
'prob2' ,
'probtie' ,
'proj_score1' ,
'proj_score2' ,


'team1_Atalanta' ,
'team1_Barcelona' ,
'team1_Manchester City' ,
'team1_RB Leipzig' ,
'team2_Atletico Madrid' ,
'team2_Bayern Munich' ,
'team2_Lyon' ,
'team2_Paris Saint-Germain']]

In [91]:

X_train.dropna().info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6168 entries, 0 to 33069
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   spi1                       6168 non-null   float64
 1   spi2                       6168 non-null   float64
 2   prob1                      6168 non-null   float64
 3   prob2                      6168 non-null   float64
 4   probtie                    6168 non-null   float64
 5   proj_score1                6168 non-null   float64
 6   proj_score2                6168 non-null   float64
 7   score1                     6168 non-null   float64
 8   score2                     6168 non-null   float64
 9   team1_Atalanta             6168 non-null   uint8  
 10  team1_Barcelona            6168 non-null   uint8  
 11  team1_Manchester City      6168 non-null   uint8  
 12  team1_RB Leipzig           6168 non-null   uint8  
 13  team2_Atletico Madrid      6168 non-null   uint

In [95]:
X_train = X_train.dropna().drop(['score1','score2'],axis=1)


In [96]:
lm = LinearRegression()
lm.fit(X_train,old[['score1','score2']].dropna())
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsCL_LR2.xlsx")

4
4


In [98]:
tX_train = X_train.values
tX_test =X_test.values
y_train = old[['score1','score2']].dropna().values


In [99]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(6168, 15)

In [100]:
model = Sequential()

In [101]:
model.add(Flatten())
model.add(Dense(1024,activation=tf.nn.relu))
model.add(Dense(2))

In [102]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [103]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)


In [104]:
model.fit(tX_train,y_train,epochs=200,callbacks=[early_stop])

Train on 6168 samples
Epoch 1/200
6168/6168 [==============================] - 1s 151us/sample - loss: 1.4665
Epoch 2/200
6168/6168 [==============================] - 0s 68us/sample - loss: 1.3708
Epoch 3/200
6168/6168 [==============================] - 0s 57us/sample - loss: 1.3722
Epoch 4/200
6168/6168 [==============================] - 0s 68us/sample - loss: 1.3646
Epoch 5/200
6168/6168 [==============================] - 0s 70us/sample - loss: 1.3689
Epoch 6/200
6168/6168 [==============================] - 0s 71us/sample - loss: 1.3664
Epoch 7/200
6168/6168 [==============================] - 1s 88us/sample - loss: 1.3642
Epoch 8/200
6168/6168 [==============================] - 1s 104us/sample - loss: 1.3645
Epoch 9/200
6168/6168 [==============================] - 1s 113us/sample - loss: 1.3629
Epoch 10/200
6168/6168 [==============================] - 1s 140us/sample - loss: 1.3679
Epoch 11/200
6168/6168 [==============================] - 1s 150us/sample - loss: 1.3642
Epoch 12/200
6

In [105]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Predicted Score 1'] = score1
new['Predicted Score 2'] = score2
new.to_excel("ProjectionsCL_TF2.xlsx")

4
4
